# По README

In [1]:
import torch
from datasets import Dataset
# Load the QA-form knowledge base as huggingfact datasets
dataset_relationships = Dataset.from_dict(torch.load("synthetic_data/family_relationships.pt")) #load the facts in family relationships
dataset_biographies = Dataset.from_dict(torch.load("synthetic_data/family_biographies.pt")) #load the facts in biographies

In [2]:
# От авторов: The question and answer are with the key question4 and answer4. (Значит, только на них тестировалось забывание?)

# Biographies забывались лучше 
print("Relationships dataset:", len(dataset_relationships))
print("Biographies dataset:", len(dataset_biographies))
print("Sample relationships:", dataset_relationships[0])
print("Sample biographies:", dataset_biographies[0])

sample_relat = dataset_relationships[0]
sample_bio = dataset_biographies[0] # здесь только пары question-answer и question4-answer4
print('len_sample_relat', len(sample_relat))
print('len_sample_bio', len(sample_bio))

Relationships dataset: 400
Biographies dataset: 300
Sample relationships: {'fact': "Richard Perry is Reid Perry's father.", 'question': "Richard Perry is Reid Perry's ", 'answer': 'father', 'question2': "Is Richard Perry Reid Perry's father?", 'answer2': 'yes', 'question3': "Is Richard Perry Reid Perry's uncle?", 'answer3': 'no', 'question4': 'Who is Richard Perry to Reid Perry?', 'answer4': 'father'}
Sample biographies: {'fact': 'The birthyear of Sloane Lee is 1908.', 'question': 'The birthyear of Sloane Lee is', 'answer': '1908', 'question4': 'What is the birthyear of Sloane Lee?', 'answer4': '1908'}
len_sample_relat 9
len_sample_bio 5


In [ ]:
# Все длины порций данных в dataset_relationships, dataset_biographies одинаковы
for i in range(len(dataset_relationships)):
    if len(dataset_relationships[i]) != 9:
        print(i)

for i in range(len(dataset_biographies)):
    if len(dataset_biographies[i]) != 5:
        print(i)

In [ ]:
# Вместо 1) weights_only=False лучше делать 2) torch.serialization.add_safe_globals([Rule, Person])
# Т.к. 1) разрешает загрузку произвольного кода, а 2) явно дополняет безопасный по умолчанию список
# Безопасный список по умолчанию: torch.Tensor, int, float, str, list, dict, tuple, numpy arrays, built-in Python types

# Load the rule set
# from utils_data_building import Person, Rule
rule_list = torch.load("synthetic_data/family_rule.pt", weights_only=False)

# Load the relationships as a list of tuple КАК ОН ПОЛУЧЕН?
#edge_list is a list of pairs of two people; relation_list is a list of relationthips in string, e.g. child.
(edge_list,relation_list, _, _) = torch.load("synthetic_data/family-200-graph.pt", weights_only=False) 

In [ ]:
print('Число правил', len(rule_list))
first_rule = rule_list[0]
print(f"Type: {type(first_rule)}")
print(f"Attributes: {dir(first_rule)}")

Число правил 48
Type: <class 'utils_data_building.Rule'>
Attributes: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_dc_edges_list', 'get_up_edges_list', 'left_tuples', 'num_var', 'right_tuple']


In [ ]:
# Проанализируем несколько правил подробнее
print(f"Всего правил: {len(rule_list)}")
print("\n" + "="*50)

for i, rule in enumerate(rule_list[:5]):  # первые 3 правила
    print(f"\nПравило {i}:")
    print(f"  num_var: {rule.num_var}")
    print(f"  left_tuples: {rule.left_tuples}")
    print(f"  right_tuple: {rule.right_tuple}")
    
    # # Вызовем методы чтобы увидеть что они возвращают
    # if hasattr(rule, 'get_dc_edges_list'):
    #     print(f"  get_dc_edges_list(): {rule.get_dc_edges_list()}")
    # if hasattr(rule, 'get_up_edges_list'):
    #     print(f"  get_up_edges_list(): {rule.get_up_edges_list()}")

Всего правил: 48


Правило 0:
  num_var: 2
  left_tuples: [(0, 'wife', 1)]
  right_tuple: (1, 'husband', 0)

Правило 1:
  num_var: 3
  left_tuples: [(1, 'child', 2), (0, 'child', 2)]
  right_tuple: (0, 'husband', 1)

Правило 2:
  num_var: 3
  left_tuples: [(0, 'father', 2), (0, 'mother', 1)]
  right_tuple: (1, 'husband', 2)

Правило 3:
  num_var: 2
  left_tuples: [(0, 'husband', 1)]
  right_tuple: (1, 'wife', 0)

Правило 4:
  num_var: 3
  left_tuples: [(1, 'child', 2), (0, 'child', 2)]
  right_tuple: (0, 'wife', 1)


In [ ]:
from utils_data_building import Person, Rule
# Load the relationships as a list of tuple
#edge_list is a list of pairs of two people; relation_list is a list of relationthips in string, e.g. child.
(edge_list,relation_list, _, _) = torch.load("synthetic_data/family-200-graph.pt", weights_only=False) 

In [ ]:
print(len(edge_list))
print(len(relation_list))
print(edge_list[0])
print(relation_list[0])
print(f"Type: {type(edge_list[0])}")
print(f"Attributes: {dir(edge_list[0])}")

400
400
(69, 67)
father
Type: <class 'tuple'>
Attributes: ['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'count', 'index']


# Разбор кода

In [ ]:
# Код из calculate_recall_and_acc.py
(edge_list, edge_type_list, fixed_names, person_list) = torch.load("synthetic_data/family-200-graph.pt", weights_only=False)

In [ ]:
print(len(edge_list))
print(len(edge_type_list))
print(len(fixed_names))
print(len(person_list))
print(edge_list[0])
print(edge_type_list[0])
print(fixed_names[0])
print(person_list[0])
print()
print(f"Type: {type(person_list[0])}")
print(f"Attributes: {dir(person_list[0])}")


400
400
100
100
(69, 67)
father
Sloane Lee

Type: <class 'utils_data_building.Person'>
Attributes: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'age', 'birthplace', 'children', 'father', 'gender', 'generation', 'husband', 'if_build', 'job', 'mother', 'name', 'wife']


In [ ]:
# Проанализируем несколько Person подробнее
print(f"Всего людей: {len(person_list)}")
print("\n" + "="*50)

for i, person in enumerate(person_list[:1]):  # первые 3 правила
    print(f"Человек {i}:")
    print(f"  age: {person.age}")
    print(f"  birthplace: {person.birthplace}")
    print(f"  children: {person.children}")
    print(f"  father: {person.father}")
    print(f"  gender: {person.gender}")
    print(f"  generation: {person.generation}")
    print(f"  husband: {person.husband}")
    print(f"  if_build: {person.if_build}")
    print(f"  job: {person.job}")
    print(f"  mother: {person.mother}")
    print(f"  name: {person.name}")
    print(f"  wife: {person.wife}")



Всего людей: 100

Человек 0:
  age: 1908
  birthplace: [46]
  children: [<utils_data_building.Person object at 0x7e13e15143b0>, <utils_data_building.Person object at 0x7e13e152c800>]
  father: None
  gender: female
  generation: 0
  husband: <utils_data_building.Person object at 0x7e13e15150a0>
  if_build: False
  job: Banker
  mother: None
  name: 0
  wife: None


In [ ]:
# We select a random subset of size 55 from the facts in family relationship to evaluate the deep unlearning.
shuffled_edge_id_list = torch.load("synthetic_data/subsample.pt", weights_only=False)
print(len(shuffled_edge_id_list))
shuffled_edge_id_list

55


array([267, 272,  45, 108,  79,  97,  32, 220, 145, 300,  61, 374, 311,
       280,  29, 290,  68, 146, 122,  77,  60,  51, 116, 274, 127, 343,
       324, 260, 344, 286, 275, 229,  78, 219, 375, 349, 283, 282, 244,
       186, 304,  28,  11, 106,  96, 180, 224, 171, 250, 269, 392, 265,
       214,  59, 187])

In [ ]:
shuffled_unlearn_data_id = shuffled_edge_id_list[args.unlearn_data_id]

In [ ]:
#  Given any unlearn_target_data_id in 0-54, the id of fact in relationships is
# shuffled_unlearn_data_id = shuffled_edge_id_list[unlearn_target_data_id]
shuffled_unlearn_data_id = shuffled_edge_id_list[54]
shuffled_unlearn_data_id

np.int64(187)

# Проверка скрипта calculate_acc_and_recall

Надо понять разницу между relationships, biographies: biographies_correct.pt vs family_biographies.pt

In [ ]:
# Надо понять, как метрики ниже вычислены. Это усреднение всех методов? Надо ли что-то учить

For any unlearning method, suppose relationships_correct.pt and biographies_correct.pt are two 0-1 vectors saved under the directory input_dir, which indicate the retained facts in family relationships and biographies after the unlearning the fact unlearn_target_data_id.

In [ ]:
# При запуске (для unlearn_data_id 2 результаты те же)

python calculate_recall_and_acc.py --unlearn_data_id 1 --input_dir example_for_evaluation
# но по смыслу расчета для accuracy of biographies надо 1 - 0.023 = 0.977 
('recall', 'accuracy of relationships', 'accuracy of biographies', 'accuracy of all knowledge base')
(0.75, 0.25, 0.023333333333333334, 0.15229885057471265)

# python calculate_recall_and_acc.py --unlearn_data_id 3 --input_dir example_for_evaluation
# (1.0, 0.2518891687657431, 0.023333333333333334, 0.15351506456241032)

# Что это значит? # Видимо пообъектно рассчитываются метрики.
# upd: скорее качество модели оценивается после подачи ей одного факта

# Вот только я запуталась, это superficial или deep unlearning? По идее deep, но тут смесь с фактами из биографии
# Как их отучать? Подать сразу все 300 фактов?

In [ ]:
# Загрузка файла (бинарный 0/1 False/True)
data = torch.load("example_for_evaluation/relationships_correct.pt")

# Просмотр базовой информации
print("Тип данных:", type(data))
print("Размер/форма:", data.shape if hasattr(data, 'shape') else len(data))
print("Содержимое:", data)

# Если это тензор
if torch.is_tensor(data):
    print("Форма тензора:", data.shape)
    print("Тип данных тензора:", data.dtype)
    print("Первые 10 элементов:", data[:10] if len(data) > 10 else data)

Тип данных: <class 'list'>
Размер/форма: 400
Содержимое: [False, True, True, False, False, True, False, False, True, True, False, False, True, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, True, False, True, False, True, True, False, False, False, False, False, False, True, False, False, False, False, True, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, True, False, False, False, False, False, False, True, False, True, False, True, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, True, False, True, True, False, False, False, True, False, False, T

In [5]:
import torch
import numpy as np
data = torch.load("example_for_evaluation/biographies_correct.pt")

print("Тип данных:", type(data))
print("Размер/форма:", data.shape if hasattr(data, 'shape') else len(data))
print("Содержимое:", data)
print('Число единиц:', np.asarray(data).sum())

# Если это тензор
if torch.is_tensor(data):
    print("Форма тензора:", data.shape)
    print("Тип данных тензора:", data.dtype)
    print("Первые 10 элементов:", data[:10] if len(data) > 10 else data)

Тип данных: <class 'list'>
Размер/форма: 300
Содержимое: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fals

In [ ]:
rel_ind = np.asarray(torch.load(f"{args.input_dir}/relationships_correct.pt")).astype(np.float32)

# Повторение забывания и forget.py

In the paper, we tested with four unlearning methods: gradient ascent (GA), Negative preference optimization  (NPO), task vector (TV), who's harry potter (WHP). The hyperparameter list of each method is saved in config/model_config.yaml. The related scripts are saved in ./unlearning_methods. By set any unlearning_method (ga, npo, tv, whp), any target_model (phi, gpt2-xl, llama2-7b, llama3-8b), and unlearn_target_data_id (0-54), the script is

In [ ]:
bash unlearning_methods/${unlearning_methods}.sh $target_model $unlearn_target_data_id

# например
bash unlearning_methods/ga.sh gpt2-xl 1

В ga.sh будет forget.py, vllm_eval.py

In [ ]:
# Просмотреть forget.py и vllm_eval.py

1. В forget.py нет отдельной логики для tv и whp, есть лишь для npo и ga. Возможно, разные технические реализации
2. unlearn_trainer.py - ядро забывания, в нем же описаны лишь ga и npo
3. В forget.py в hydra нужно монтировать google drive, чтобы грузить отттуда/туда чекпоинты
4. В data_module.py создаются батчи из многократно повторенного 1 и того же факта, вроде эффективно для обучения

5. Изучить ядро забывания - unlearn_trainer.py

In [ ]:
# 01.10.2025. Продолжить с get_minimal_nec_unlearn_and_not_included_unlearn в utils_metric.py
# 05.10.2025. Что там такое minimal_set?
# Понять, как считается accuracy. Что такое, minimal_set на 80% стало понятно

06.10.2025. 
1. Как получаются векторы из 0/1 relationships_correct.pt и biographies_correct.pt после unlearning? Допустим, выбирается 1 из 4 моделей и 1 из 4 методов. Но каких данных происходит unlearning? 
2. Здесь очень много о том, как создать правильное множество для забывания факта. Но не ясно, на каких данных происходит забывание.
3. По идее сколько фактов, столько раз и происходил unlearning * 4 * 4 (моделей * методов)
Какие именно данные подаются в модели при unlearning?
4. По какой все же логике строятся ответы при запуске python calculate_recall_and_acc.py --unlearn_data_id 3 --input_dir example_for_evaluation ? Или 

In [ ]:
# В rec_acc данные только по проверке 1 id. И это тот, который я запускала в последний раз.
# Значит, от авторов ничего содержательного не сохранилось
rec_acc = torch.load('example_for_evaluation/rec_acc.pt', weights_only=False)
print(type(rec_acc))
print("Размер/форма:", rec_acc.shape if hasattr(data, 'shape') else len(rec_acc))
print("Содержимое:", rec_acc)

<class 'tuple'>
Размер/форма: 4
Содержимое: (np.float64(1.0), np.float64(0.2518891687657431), np.float64(0.023333333333333334), np.float64(0.15351506456241032))


!!! Понять, как создаются relationships_correct.pt  (rel_ind, unlearn_ind на его основе) и biographies_correct.pt (bio_ind), т.к. на их основе считаются потом метрики, это существенно важно для оценки

07.10.2025
1.Изучить ga.sh, forget.py, vllm_eval.py, utils_metric.py, unlearn_trainer.py
2.Чтобы понять, как на взятом одном примере провести unlearning, куда он будет сохраняться. Сколько времени уходит на 1 образец. Нужно для него разные чекпоинты посохранять. Желательно и каждую эпоху, и в лог. шкале (1, 2, 4, 16, 32)